# Classifiers
### Example: Built-in vs. Generic Classifiers

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from quantopian.pipeline import Pipeline
from quantopian.pipeline.factors import Returns
from quantopian.research import run_pipeline

In [2]:
from quantopian.pipeline.data.morningstar import asset_classification, valuation
from quantopian.pipeline.classifiers.morningstar import Sector

# These produce teh same data, but Sector has symbolic constants and hand-written docs:
sector_generic = asset_classification.morningstar_sector_code
sector_builtin = Sector()

In [3]:
sector_generic

asset_classification<US>.morningstar_sector_code::int64

In [4]:
sector_builtin

Sector([asset_classification<US>.morningstar_sector_code], 1)

In [5]:
print("Docs for built-in Sector class:\n" + sector_builtin.__doc__)

Docs for built-in Sector class:

        Classifier that groups assets by Morningstar Sector Code.

        There are 11 possible classifications:

        * 101 - Basic Materials
        * 102 - Consumer Cyclical
        * 103 - Financial Services
        * 104 - Real Estate
        * 205 - Consumer Defensive
        * 206 - Healthcare
        * 207 - Utilities
        * 308 - Communication Services
        * 309 - Energy
        * 310 - Industrials
        * 311 - Technology

        These values are provided as integer constants on the class.

        For more information on morningstar classification codes, see:
        https://www.quantopian.com/help/fundamentals#industry-sector.
        


In [6]:
print "Symbolic Constants:"
dir(sector_builtin)[1:12]

Symbolic Constants:


['BASIC_MATERIALS',
 'COMMUNICATION_SERVICES',
 'CONSUMER_CYCLICAL',
 'CONSUMER_DEFENSIVE',
 'ENERGY',
 'FINANCIAL_SERVICES',
 'HEALTHCARE',
 'INDUSTRIALS',
 'REAL_ESTATE',
 'SECTOR_NAMES',
 'TECHNOLOGY']

In [7]:
from quantopian.pipeline.data.morningstar import valuation_ratios
# https://www.quantopian.com/posts/pipeline-classifiers-are-here

def grouped_earnings_yield_example():
    sector = Sector()
    # earning_yield = valuation_ratios.earning_yield.latest
    
    # zscored_naive = earning_yield.zscore()
    # zscored_grouped = earning_yield.zscore(groupby=sector)
    return Pipeline(
        columns={
            'sector': sector,
            # 'yield': earning_yield,
            #'yield_zscored': zscored_naive,
            #'yield_zscored_grouped': zscored_grouped,
        },
        # screen=zscored_grouped.notnull(),
    )
    
yields = run_pipeline(grouped_earnings_yield_example(), '2019-07-26', '2019-07-26')
yields.head()

sector
2019-07-26 00:00:00+00:00 Equity(2 [ARNC])         310
                          Equity(21 [AAME])        103
                          Equity(24 [AAPL])        311
                          Equity(25 [ARNC_PR])     101
                          Equity(41 [ARCB])        310

In [8]:
sector_df = yields.sector.unstack().fillna(0)
sector_df

,Equity(2 [ARNC]),Equity(21 [AAME]),Equity(24 [AAPL]),Equity(25 [ARNC_PR]),Equity(41 [ARCB]),Equity(52 [ABM]),Equity(53 [ABMD]),Equity(62 [ABT]),Equity(64 [GOLD]),Equity(66 [AB]),...,Equity(53369 [CNBS]),Equity(53370 [HCAT]),Equity(53371 [PDEV]),Equity(53372 [PLC]),Equity(53373 [PSM]),Equity(53374 [CSTL]),Equity(53379 [LVGO]),Equity(53380 [TOKE]),Equity(53381 [RCB]),Equity(53382 [FLLC_U])
2019-07-26 00:00:00+00:00,310,103,311,101,310,310,206,206,101,103,...,-1,311,-1,-1,-1,206,206,-1,-1,310


### Quiz 1
How many unique sectors are in the yields variable?

In [9]:
from scipy.stats import itemfreq
itemfreq(yields.sector)

array([[  -1, 2728],
       [ 101,  297],
       [ 102,  603],
       [ 103, 1531],
       [ 104,  453],
       [ 205,  219],
       [ 206,  908],
       [ 207,  138],
       [ 308,   90],
       [ 309,  317],
       [ 310,  813],
       [ 311,  732]])

In [10]:
unique, counts = np.unique(yields.sector, return_counts = True)
print(np.asarray((unique, counts)).T)

[[  -1 2728]
 [ 101  297]
 [ 102  603]
 [ 103 1531]
 [ 104  453]
 [ 205  219]
 [ 206  908]
 [ 207  138]
 [ 308   90]
 [ 309  317]
 [ 310  813]
 [ 311  732]]


In [11]:
len(unique)

12

In [12]:
len(counts)

12

In [13]:
yields.shape

(8829, 1)

In [14]:
from quantopian.pipeline.factors import AverageDollarVolume
from quantopian.pipeline.filters.morningstar import IsPrimaryShare

def masked_zscore_returns_example():
    returns = Returns(window_length=252)
    
    is_liquid = AverageDollarVolume(window_length=30).percentile_between(75, 100)
    is_primary = IsPrimaryShare()
    no_returns_outliers = returns.percentile_between(2, 98)
    base_universe = is_liquid & no_returns_outliers & is_primary
    
    masked_zscored = returns.zscore(mask=base_universe)
    
    return Pipeline(
        columns={'1YearReturns': returns}, 
        screen=masked_zscored.notnull()
    )

results2 = run_pipeline(masked_zscore_returns_example(), '2017-07-26', '2019-07-26')
results2.head()

1YearReturns
2017-07-26 00:00:00+00:00 Equity(2 [ARNC])      -0.179684
                          Equity(24 [AAPL])      0.610342
                          Equity(53 [ABMD])      0.297803
                          Equity(62 [ABT])       0.194298
                          Equity(67 [ADSK])      0.911319

In [15]:
returns_df = results2['1YearReturns'].unstack().fillna(0)
returns_df

,Equity(2 [ARNC]),Equity(24 [AAPL]),Equity(31 [ABAX]),Equity(41 [ARCB]),Equity(52 [ABM]),Equity(53 [ABMD]),Equity(62 [ABT]),Equity(66 [AB]),Equity(67 [ADSK]),Equity(76 [TAP]),...,Equity(52084 [VNE]),Equity(52100 [AVLR]),Equity(52121 [EPRT]),Equity(52144 [LOVE]),Equity(52159 [BJ]),Equity(52165 [DOMO]),Equity(52200 [RPAY]),Equity(52209 [ALLK]),Equity(52211 [TLRY]),Equity(52233 [BE])
2017-07-26 00:00:00+00:00,-0.179684,0.610342,0.0,0.0,0.000000,0.297803,0.194298,0.0,0.911319,-0.082025,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2017-07-27 00:00:00+00:00,-0.196155,0.519862,0.0,0.0,0.000000,0.251034,0.182805,0.0,0.914108,-0.027925,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2017-07-28 00:00:00+00:00,-0.205803,0.470509,0.0,0.0,0.000000,0.243652,0.155659,0.0,0.876682,-0.068139,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2017-07-31 00:00:00+00:00,-0.206236,0.462187,0.0,0.0,0.000000,0.239702,0.136459,0.0,0.875526,-0.113254,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2017-08-01 00:00:00+00:00,-0.201164,0.429557,0.0,0.0,0.000000,0.238645,0.113174,0.0,0.854728,-0.102743,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2017-08-02 00:00:00+00:00,-0.178297,0.463801,0.0,0.0,0.000000,0.234621,0.117307,0.0,0.924659,-0.100695,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2017-08-03 00:00:00+00:00,-0.193450,0.513793,0.0,0.0,0.000000,0.234610,0.122259,0.0,0.883490,-0.062769,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2017-08-04 00:00:00+00:00,-0.177343,0.488850,0.0,0.0,0.000000,0.218040,0.123206,0.0,0.852017,-0.074945,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2017-08-07 00:00:00+00:00,-0.192261,0.474601,0.0,0.0,0.000000,0.256363,0.115271,0.0,0.826871,-0.071113,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2017-08-08 00:00:00+00:00,-0.182619,0.485271,0.0,0.0,0.000000,0.275237,0.122004,0.0,0.849250,-0.077038,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [16]:
from quantopian.pipeline.factors import AverageDollarVolume
from quantopian.pipeline.filters.morningstar import IsPrimaryShare

def masked_zscore_returns_example():
    returns = Returns(window_length=252)
    sector = Sector()
    
    is_liquid = AverageDollarVolume(window_length=30).percentile_between(75, 100)
    is_primary = IsPrimaryShare()
    no_returns_outliers = returns.percentile_between(2, 98)
    base_universe = is_liquid & no_returns_outliers & is_primary
    
    masked_zscored = returns.demean(groupby=Sector()).zscore(mask=base_universe)
    
    return Pipeline(
        columns={'1YearReturns': returns, 'sector': sector}, 
        screen=masked_zscored.notnull()
    )

results2 = run_pipeline(masked_zscore_returns_example(), '2017-07-26', '2019-07-26')
results2.head()

1YearReturns  sector
2017-07-26 00:00:00+00:00 Equity(2 [ARNC])      -0.179684     310
                          Equity(24 [AAPL])      0.610342     311
                          Equity(53 [ABMD])      0.297803     206
                          Equity(62 [ABT])       0.194298     206
                          Equity(67 [ADSK])      0.911319     311

### Ranked returns

In [28]:
from quantopian.pipeline.factors import AverageDollarVolume
from quantopian.pipeline.filters.morningstar import IsPrimaryShare

def masked_zscore_returns_example():
    returns = Returns(window_length=252)
    sector = Sector()
    
    is_liquid = AverageDollarVolume(window_length=30).percentile_between(25, 100)
    is_primary = IsPrimaryShare()
    no_returns_outliers = returns.percentile_between(2, 98)
    base_universe = is_liquid & no_returns_outliers & is_primary
    
    masked_zscored = returns.demean(groupby=Sector()).zscore(mask=base_universe)
    ranked_return = returns.rank()
    
    return Pipeline(
        columns={'1YearReturns': returns, 'sector': sector, 'ranked_return': ranked_return}, 
        screen=masked_zscored.notnull()
    )

results2 = run_pipeline(masked_zscore_returns_example(), '2017-07-26', '2019-07-26')
results2.head()

1YearReturns  ranked_return  \
2017-07-26 00:00:00+00:00 Equity(2 [ARNC])      -0.179684          946.0   
                          Equity(24 [AAPL])      0.610342         6456.0   
                          Equity(31 [ABAX])      0.076093         3104.0   
                          Equity(41 [ARCB])      0.265384         5160.0   
                          Equity(52 [ABM])       0.201596         4641.0   

                                             sector  
2017-07-26 00:00:00+00:00 Equity(2 [ARNC])      310  
                          Equity(24 [AAPL])     311  
                          Equity(31 [ABAX])     206  
                          Equity(41 [ARCB])     310  
                          Equity(52 [ABM])      310

In [29]:
results2 = results2.sort_values(by=['ranked_return'], ascending = False)

In [30]:
results2.filter(like='2019-07-26', axis = 0).head(50)

1YearReturns  ranked_return  \
2019-07-26 00:00:00+00:00 Equity(22364 [EXAS])      0.854083         7097.0   
                          Equity(50967 [SAFE])      0.852824         7096.0   
                          Equity(6449 [RGEN])       0.852710         7095.0   
                          Equity(52115 [EIDX])      0.851832         7094.0   
                          Equity(44757 [NRC])       0.846499         7093.0   
                          Equity(40399 [IPHI])      0.844657         7092.0   
                          Equity(51746 [VCTR])      0.843233         7091.0   
                          Equity(46498 [REPH])      0.828678         7088.0   
                          Equity(35259 [LRN])       0.809294         7087.0   
                          Equity(24827 [RCII])      0.807875         7086.0   
                          Equity(32187 [ATEC])      0.803571         7085.0   
                          Equity(6683 [SBUX])       0.803505         7084.0   
                          Equity(47418 [ENLV])      0.801724         7083.0   
                          Equity(18588 [MANH])      0.797150         7082.0   
                          Equity(21893 [ISSC])      0.795987         7081.0   
                          Equity(16437 [VCEL])      0.790521         7080.0   
                          Equity(48567 [ATEX])      0.790099         7079.0   
                          Equity(34525 [MELI])      0.788014         7078.0   
                          Equity(34118 [PRO])       0.787324         7077.0   
                          Equity(16453 [CIEN])      0.773416         7076.0   
                          Equity(18183 [MRCY])      0.760595         7075.0   
                          Equity(12898 [HLIT])      0.759091         7074.0   
                          Equity(7822 [UQM])        0.757895         7073.0   
                          Equity(49189 [LNTH])      0.744906         7072.0   
                          Equity(393 [AMSC])        0.734818         7071.0   
                          Equity(36209 [IOVA])      0.727402         7070.0   
                          Equity(52128 [IIIV])      0.726146         7069.0   
                          Equity(49204 [CABO])      0.724337         7068.0   
                          Equity(40848 [FLDM])      0.722552         7067.0   
                          Equity(28016 [CMG])       0.720944         7066.0   
                          Equity(24474 [CLCT])      0.719015         7065.0   
                          Equity(45238 [FOXF])      0.719000         7064.0   
                          Equity(9638 [RAVE])       0.712903         7063.0   
                          Equity(45080 [MRTX])      0.709666         7062.0   
                          Equity(26781 [MKTX])      0.709612         7061.0   
                          Equity(51962 [GSHD])      0.709540         7060.0   
                          Equity(10417 [ARWR])      0.703967         7058.0   
                          Equity(51535 [ODT])       0.689202         7056.0   
                          Equity(48518 [ASND])      0.684882         7055.0   
                          Equity(4370 [LAWS])       0.684874         7054.0   
                          Equity(47346 [NBEV])      0.681818         7053.0   
                          Equity(34560 [CZZ])       0.681604         7052.0   
                          Equity(2837 [FICO])       0.678594         7051.0   
                          Equity(48926 [XBIT])      0.672811         7050.0   
                          Equity(30301 [NEO])       0.671377         7049.0   
                          Equity(34238 [DHX])       0.666667         7048.0   
                          Equity(46569 [PCTY])      0.660499         7047.0   
                          Equity(3950 [INT])        0.657300         7046.0   
                          Equity(1884 [CRVL])       0.656036         7045.0   
                          Equity(3695 [HSY])        0.655791         7044.0   

                  